In [1]:
import numpy as np
import xroms
from glob import glob
import matplotlib.pyplot as plt
import xarray as xr
import pandas as pd
import xrft as xrft
import warnings
import sys
import xrft
sys.path.append("/homes/metogra/iufarias/FeedbackSubmeso/useful/")
import romspickle

from dask.diagnostics import ProgressBar
import dask
from dask.distributed import Client, LocalCluster

import scipy.integrate as integ
# warnings.filterwarnings("ignore")
import seawater as sw
from scipy.interpolate import interp1d

In [2]:
#example to use dask
# import dask
# from dask import delayed

# def executeCalc(ret):
#     lat,lon,level,tmp = ret
#     potempList = []
#     for i in range (0,tmp.shape[0]):
#         tmpInstant = tmp[i,:,:,:]
#         potempList.append(delayed(pot)(tmpInstant,lat,lon,level))
#     results = dask.compute(potempList)

In [3]:
#plot specs

plt.rcParams['text.usetex'] = False
plt.rcParams.update({'font.size': 20})

plt.rcParams['font.family'] = 'sans-serif'
plt.rcParams['font.sans-serif'] = ['Lucida Grande']


In [4]:
# client = Client(processes=False)
client = Client(n_workers=15,processes=True)
client

2024-08-19 18:39:44,047 - distributed.diskutils - INFO - Found stale lock file and directory '/tmp/dask-worker-space/worker-0iy2ca0z', purging
2024-08-19 18:39:44,047 - distributed.diskutils - INFO - Found stale lock file and directory '/tmp/dask-worker-space/worker-hbtmmkop', purging
2024-08-19 18:39:44,047 - distributed.diskutils - INFO - Found stale lock file and directory '/tmp/dask-worker-space/worker-r4hisv4e', purging
2024-08-19 18:39:44,047 - distributed.diskutils - INFO - Found stale lock file and directory '/tmp/dask-worker-space/worker-vtk2vdno', purging
2024-08-19 18:39:44,048 - distributed.diskutils - INFO - Found stale lock file and directory '/tmp/dask-worker-space/worker-zu7zcjhe', purging
2024-08-19 18:39:44,048 - distributed.diskutils - INFO - Found stale lock file and directory '/tmp/dask-worker-space/worker-ck5kd3_5', purging
2024-08-19 18:39:44,048 - distributed.diskutils - INFO - Found stale lock file and directory '/tmp/dask-worker-space/worker-vf90f88q', purging

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:8787/status,
Dashboard: http://127.0.0.1:8787/status,Workers: 15
Total threads: 135,Total memory: 502.98 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:39127,Workers: 15
Dashboard: http://127.0.0.1:8787/status,Total threads: 135
Started: Just now,Total memory: 502.98 GiB
Comm: tcp://127.0.0.1:35807,Total threads: 9
Dashboard: http://127.0.0.1:34501/status,Memory: 33.53 GiB
Nanny: tcp://127.0.0.1:35715,


2024-08-20 23:25:21,775 - distributed.nanny - WARNING - Restarting worker
2024-08-20 23:25:22,006 - distributed.nanny - WARNING - Restarting worker
2024-08-20 23:25:22,163 - distributed.nanny - WARNING - Restarting worker
2024-08-20 23:25:22,167 - distributed.nanny - WARNING - Restarting worker
2024-08-20 23:25:22,296 - distributed.nanny - WARNING - Restarting worker
2024-08-20 23:25:22,460 - distributed.nanny - WARNING - Restarting worker
2024-08-20 23:25:22,611 - distributed.nanny - WARNING - Restarting worker
2024-08-20 23:25:22,776 - distributed.nanny - WARNING - Restarting worker
2024-08-20 23:25:22,781 - distributed.nanny - WARNING - Restarting worker
2024-08-20 23:25:22,944 - distributed.nanny - WARNING - Restarting worker
2024-08-20 23:25:23,260 - distributed.nanny - WARNING - Restarting worker
2024-08-20 23:25:23,424 - distributed.nanny - WARNING - Restarting worker
2024-08-20 23:25:23,431 - distributed.nanny - WARNING - Restarting worker
2024-08-20 23:25:23,599 - distributed.

In [5]:
# chunks={'time_counter':10, 'x_rho':50, 'y_rho':190,'s_rho':80,'s_w':80}

# chunks={'time_counter':1,'x_rho':1000,"x_u":1000,"x_v":1000,
#                         'y_rho':1520,"y_u":1520,"y_v":1520,'s_rho':80,'s_w':80}

chunks={'time_counter':1,'x_rho':100,"x_u":100,"x_v":100,
                        'y_rho':152,"y_u":152,"y_v":152,'s_rho':80,'s_w':80}


# chunks={'time_counter':1,'x_rho':1000,"x_u":1000,"x_v":1000,
#                         'y_rho':1520,"y_u":1520,"y_v":1520,'s_rho':80,'s_w':80}

# M='01'
path='/data/pacific/lrenault/SASI/CROCO/'


# varf=xr.open_mfdataset(path + 'FULL/SASI*2012-{0}*.nc'.format(M),data_vars='minimal')
# vart=xr.open_mfdataset(path + 'SMTH/SASI*2012-{0}*.nc'.format(M),data_vars='minimal')

varf=xr.open_mfdataset(path + 'FULL/SASI*2012*.nc',data_vars='minimal',chunks=chunks)
vart=xr.open_mfdataset(path + 'SMTH/SASI*2012*.nc',data_vars='minimal',chunks=chunks)

In [6]:
varx = varf.rename({"time_counter":"ocean_time","x_rho": "xi_rho","x_u":"xi_u","x_v":"xi_v","x_w":"xi_w",
                    "y_rho":"eta_rho","y_v":"eta_v","y_u":"eta_u","y_w":"eta_w","s_rho":"s_rho","s_w":"s_w"})
df,gf=xroms.roms_dataset(varx,Vtransform=varx.Vtransform.data)

vary = vart.rename({"time_counter":"ocean_time","x_rho": "xi_rho","x_u":"xi_u","x_v":"xi_v",
                    "y_rho":"eta_rho","y_v":"eta_v","y_u":"eta_u","y_w":"eta_w","s_rho":"s_rho","s_w":"s_w"})
dt,gt=xroms.roms_dataset(vary,Vtransform=vary.Vtransform.data)

/tmp/ipykernel_366718/2400411198.py:1: UserWarning: rename 's_rho' to 's_rho' does not create an index anymore. Try using swap_dims instead or use set_index after rename to create an indexed coordinate.
  varx = varf.rename({"time_counter":"ocean_time","x_rho": "xi_rho","x_u":"xi_u","x_v":"xi_v","x_w":"xi_w",
/tmp/ipykernel_366718/2400411198.py:1: UserWarning: rename 's_w' to 's_w' does not create an index anymore. Try using swap_dims instead or use set_index after rename to create an indexed coordinate.
  varx = varf.rename({"time_counter":"ocean_time","x_rho": "xi_rho","x_u":"xi_u","x_v":"xi_v","x_w":"xi_w",
/tmp/ipykernel_366718/2400411198.py:5: UserWarning: rename 's_rho' to 's_rho' does not create an index anymore. Try using swap_dims instead or use set_index after rename to create an indexed coordinate.
  vary = vart.rename({"time_counter":"ocean_time","x_rho": "xi_rho","x_u":"xi_u","x_v":"xi_v",
/tmp/ipykernel_366718/2400411198.py:5: UserWarning: rename 's_w' to 's_w' does not c

In [7]:
%time
rho_full=df.rho[:,:,120:-120,120:800]+df.rho0
rho_smth=dt.rho[:,:,120:-120,120:800]+dt.rho0

CPU times: user 4 µs, sys: 1 µs, total: 5 µs
Wall time: 6.68 µs


In [8]:
zSurf=rho_full.z_rho0.compute().max()
zBott=rho_full.z_rho0.compute().min()

In [9]:
z_surf_interp=-2
dz=10
z_interp=np.linspace(z_surf_interp,zBott, np.int(np.abs(zBott/dz)))
print('dz={0:.1f}'.format(np.diff(z_interp).mean()) +  '   n_z={0:.0f}'.format(np.int(np.abs(zBott/dz))))

dz=-10.0   n_z=507


/tmp/ipykernel_366718/553834454.py:3: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  z_interp=np.linspace(z_surf_interp,zBott, np.int(np.abs(zBott/dz)))
/tmp/ipykernel_366718/553834454.py:4: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional infor

### Setting $dx,dy,dz,dA,dV$

In [10]:
d_field=rho_full[0,:,:,:].xroms.isoslice(z_interp).compute()

DeltaX=(d_field*0+500).compute()
DeltaY=(d_field*0+500).compute()
DeltaZ=xr.apply_ufunc(np.gradient,d_field['z_rho'],kwargs={'axis':0})
DeltaA=DeltaX*DeltaY
DeltaV=np.abs(DeltaA*DeltaZ)

V=DeltaV.sum()
Az=DeltaA.sum(['eta_rho','xi_rho'])

In [11]:
%%time
# for i_x in range(rho_full.shape[0]):
for i_x in range(329,rho_full.shape[0]):
    
# for i_x in range(5):

    rho_interp=rho_full[i_x,:,:,:].xroms.isoslice(z_interp).expand_dims('ocean_time').compute()
    # rho_interp=rho_full.isel(ocean_time=slice(i_x,i_x+1)).xroms.isoslice(z_interp).compute()

    rho_flat_nonan=rho_interp.stack(ravel=['ocean_time','xi_rho','eta_rho','z_rho']).dropna(dim='ravel')

    count, bins_count=np.histogram(rho_flat_nonan,bins=1000)

    P_rho=count/sum(count)

    A_Az=integ.cumtrapz(Az[::-1],dx=np.diff(-z_interp)[0])[::-1]
    V_PDF=(np.cumsum(P_rho[::-1])*V.data)[::-1]
    b_v_PDF=(bins_count[1:]+bins_count[:-1])/2

    rho_zr=[]
    zr=[]

    for k in range(V_PDF.shape[0]):
        rho_zr.append(b_v_PDF[k])
        z_i=np.argmin(np.abs((A_Az-V_PDF[k])))
        zr.append(z_interp[z_i])

    Zr=np.array(zr)
    Rho_zr=np.array(rho_zr)
    FF = interp1d(Rho_zr,Zr,fill_value='extrapolate')

    z_star_full_d=rho_interp.copy(data=FF(rho_interp))
    del z_star_full_d.attrs['grid']
    
    z_star_full_d.to_netcdf('/data/pacific/iufarias/tseng_z_star/z_star_full_{0:03d}.nc'.format(i_x))

CPU times: user 3h 57min 54s, sys: 1h 53min 12s, total: 5h 51min 7s
Wall time: 9h 3min 22s


In [12]:
%%time
for i_x in range(rho_smth.shape[0]):
# for i_x in range(2):

    rho_interp=rho_smth[i_x,:,:,:].xroms.isoslice(z_interp).expand_dims('ocean_time').compute()
    # rho_interp=rho_full.isel(ocean_time=slice(i_x,i_x+1)).xroms.isoslice(z_interp).compute()

    rho_flat_nonan=rho_interp.stack(ravel=['ocean_time','xi_rho','eta_rho','z_rho']).dropna(dim='ravel')

    count, bins_count=np.histogram(rho_flat_nonan,bins=1000)

    P_rho=count/sum(count)

    A_Az=integ.cumtrapz(Az[::-1],dx=np.diff(-z_interp)[0])[::-1]
    V_PDF=(np.cumsum(P_rho[::-1])*V.data)[::-1]
    b_v_PDF=(bins_count[1:]+bins_count[:-1])/2

    rho_zr=[]
    zr=[]

    for k in range(V_PDF.shape[0]):
        rho_zr.append(b_v_PDF[k])
        z_i=np.argmin(np.abs((A_Az-V_PDF[k])))
        zr.append(z_interp[z_i])

    Zr=np.array(zr)
    Rho_zr=np.array(rho_zr)
    FF = interp1d(Rho_zr,Zr,fill_value='extrapolate')

    z_star_smth_d=rho_interp.copy(data=FF(rho_interp))
    del z_star_smth_d.attrs['grid']
    
    z_star_smth_d.to_netcdf('/data/pacific/iufarias/tseng_z_star/z_star_smth_{0:03d}.nc'.format(i_x))

CPU times: user 7h 15min 3s, sys: 4h 43min 38s, total: 11h 58min 41s
Wall time: 19h 34min 42s


2024-08-20 23:25:21,543 - distributed.nanny - ERROR - Worker process died unexpectedly
2024-08-20 23:25:21,543 - distributed.nanny - ERROR - Worker process died unexpectedly
2024-08-20 23:25:21,544 - distributed.nanny - ERROR - Worker process died unexpectedly
2024-08-20 23:25:21,545 - distributed.nanny - ERROR - Worker process died unexpectedly
2024-08-20 23:25:21,545 - distributed.nanny - ERROR - Worker process died unexpectedly
2024-08-20 23:25:21,545 - distributed.nanny - ERROR - Worker process died unexpectedly
2024-08-20 23:25:21,546 - distributed.nanny - ERROR - Worker process died unexpectedly
2024-08-20 23:25:21,546 - distributed.nanny - ERROR - Worker process died unexpectedly
2024-08-20 23:25:21,546 - distributed.nanny - ERROR - Worker process died unexpectedly
2024-08-20 23:25:21,546 - distributed.nanny - ERROR - Worker process died unexpectedly
2024-08-20 23:25:21,546 - distributed.nanny - ERROR - Worker process died unexpectedly
2024-08-20 23:25:21,546 - distributed.nanny